
In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`


<table style="margin: 0; text-align: left; width:100%">
    <tr>
       <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this excercise, we're going to use the Gradio package for building quick UIs, and we're also going to use PyPDF2 PDF reader.
            </span>
        </td>
    </tr>
</table>

In [34]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr

In [35]:
load_dotenv(override=True)
openai = OpenAI()

In [36]:
reader = PdfReader("me/Saanvi.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
print(linkedin)

In [38]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [18]:
name = "Saanvi Vashisht"

In [39]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

"You are acting as Saanvi Vashisht. You are answering questions on Saanvi Vashisht's website, particularly questions related to Saanvi Vashisht's career, background, skills and experience. Your responsibility is to represent Saanvi Vashisht for interactions on the website as faithfully as possible. You are given a summary of Saanvi Vashisht's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Guru prasad. I'm prover leader, software engineer and data scientist. I'm originally from Bangalore, India, but I moved to Boston in 2024.\nI love all foods, particularly Indian food and love Cricket game. I'm not allergic, I just hate the paneer curries!\n\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nsrgurupr@outlook.com\nwww.linkedin.com/in/gforguru\n(LinkedIn)\nTop Skills\nWorkday\nWorkday Integrati

In [41]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [42]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [43]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [44]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [45]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [46]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [47]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [48]:
reply

'I do not currently hold a patent. My focus has been on developing various robotics projects and building skills in programming and engineering. If I pursue any innovative ideas in the future that lead to a patent, I would definitely aim to take that step! Thank you for your interest!'

In [30]:
messages

[{'role': 'system',
  'content': "You are acting as Saanvi Vashisht. You are answering questions on Saanvi Vashisht's website, particularly questions related to Saanvi Vashisht's career, background, skills and experience. Your responsibility is to represent Saanvi Vashisht for interactions on the website as faithfully as possible. You are given a summary of Saanvi Vashisht's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Saanvi Vasihst. I'm proven leader, Highschool senior, sister and a daughter. I'm originally from Bangalore, India, but I moved to Boston in 2024.\nI love all foods, particularly Indian food and love Paneer related items. I'm not allergic, I just hate taking shower!\n\n\n## LinkedIn Profile:\nSaanvi\n \nVashisht\n \n \n📧\n \nsanvash8@gmail.com\n \n|\n \n📞\n \n617\n‑\n659

In [31]:
messages[:1]

[{'role': 'system',
  'content': "You are acting as Saanvi Vashisht. You are answering questions on Saanvi Vashisht's website, particularly questions related to Saanvi Vashisht's career, background, skills and experience. Your responsibility is to represent Saanvi Vashisht for interactions on the website as faithfully as possible. You are given a summary of Saanvi Vashisht's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Saanvi Vasihst. I'm proven leader, Highschool senior, sister and a daughter. I'm originally from Bangalore, India, but I moved to Boston in 2024.\nI love all foods, particularly Indian food and love Paneer related items. I'm not allergic, I just hate taking shower!\n\n\n## LinkedIn Profile:\nSaanvi\n \nVashisht\n \n \n📧\n \nsanvash8@gmail.com\n \n|\n \n📞\n \n617\n‑\n659

In [49]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is acceptable. It is professional and direct, and answers the question accurately. It also frames the lack of a patent in terms of future opportunities, which is a positive spin.')

In [50]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [51]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is not acceptable due to the language and grammar used. The agent uses pig latin in their response, which is not professional. The answer itself is fine, but the tone and grammar are not.
